# DL_mini project #3

In [40]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
import pandas as pd
kc_house = pd.read_csv('../../DL/data/kc_house_data.csv')

In [42]:
from sklearn.utils import shuffle
kc_house = shuffle(kc_house, random_state = 42)

In [43]:
kc_house

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
735,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,...,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
2830,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,...,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
4106,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,...,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
16218,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,...,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
19964,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,...,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,5272200045,20141113T000000,378000.0,3,1.50,1000,6914,1.0,0,0,...,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947
21575,9578500790,20141111T000000,399950.0,3,2.50,3087,5002,2.0,0,0,...,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183
5390,7202350480,20140930T000000,575000.0,3,2.50,2120,4780,2.0,0,0,...,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650
860,1723049033,20140620T000000,245000.0,1,0.75,380,15000,1.0,0,0,...,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000


In [44]:
df = kc_house.copy()
df = pd.DataFrame(kc_house)
df['year'] = pd.DatetimeIndex(df['date']).year
df = df.drop(['date','id'], axis = 1)
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
735,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014
2830,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014
4106,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014
16218,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015
19964,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,378000.0,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,2014
21575,399950.0,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,2014
5390,575000.0,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,2014
860,245000.0,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,2014


In [46]:
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
735,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014
2830,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014
4106,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014
16218,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015
19964,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,378000.0,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,2014
21575,399950.0,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,2014
5390,575000.0,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,2014
860,245000.0,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,2014


In [49]:
X_train_full = df.copy()

In [50]:
X_train_full

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
735,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014
2830,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014
4106,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014
16218,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015
19964,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,378000.0,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,2014
21575,399950.0,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,2014
5390,575000.0,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,2014
860,245000.0,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,2014


In [51]:
y_train = X_train_full[['price']]
X_train = X_train_full.drop(['price'],axis=1)

In [52]:
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
735,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014
2830,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014
4106,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014
16218,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015
19964,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,2014
21575,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,2014
5390,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,2014
860,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,2014


In [53]:
y_train

,price
735,365000.0
2830,865000.0
4106,1038000.0
16218,1490000.0
19964,711000.0
...,...
11964,378000.0
21575,399950.0
5390,575000.0
860,245000.0


In [54]:
mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std

In [60]:
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape = (X_train.shape[1],)))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(300, kernel_initializer = 'lecun_normal', activation = "selu"))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(300, kernel_initializer = 'lecun_normal', activation = "selu"))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(200, kernel_initializer = 'lecun_normal', activation = "selu"))
    model.add(keras.layers.Dense(100, kernel_initializer = 'lecun_normal', activation = "selu"))
    model.add(keras.layers.Dense(75, kernel_initializer = 'lecun_normal', activation = "selu"))
    model.add(keras.layers.Dense(50, kernel_initializer = 'lecun_normal', activation = "selu"))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1))
    
    model.compile(loss="mse", 
                  optimizer = "adam", 
                  metrics=["mse"])
    return model

In [61]:
k = 10
num_val = len(X_train) // k
num_epochs = 300
all_scores = []
temp = []

for i in range(k):
    print('processing fold #', i)
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)
    
    model = build_model()
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, 
              verbose=0)

    val_mse, val_mae = model.evaluate(X_val, y_val, verbose=0)
    all_scores.append(val_mse)
    temp.append(val_mse)    

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [62]:
all_scores

[21295417344.0,
 18522056704.0,
 18584176640.0,
 20654653440.0,
 17109914624.0,
 14494774272.0,
 15965836288.0,
 15049352192.0,
 16353064960.0,
 14243478528.0]

In [63]:
np.mean(all_scores)

17227272499.2

In [27]:
all_scores

[16869612544.0,
 14947997696.0,
 11810857984.0,
 13952116736.0,
 14038784000.0,
 11409452032.0,
 12698981376.0,
 11545284608.0,
 14699189248.0,
 11849213952.0]

In [28]:
np.mean(all_scores)

13382149017.6

In [17]:
def build_model(): 
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))
    model.add(keras.layers.Dense(300, activation='relu'))
    model.add(keras.layers.Dense(200, activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
#     model.add(keras.layers.Dense(64, activation='relu'))
#     model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
    
    return model

In [18]:
k = 10
num_val = len(X_train) // k
num_epochs = 300
all_scores = []
temp = []

for i in range(k):
    print('processing fold #', i)
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)
    
    model = build_model()
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, 
              verbose=0)

    val_mse, val_mae = model.evaluate(X_val, y_val, verbose=0)
    all_scores.append(val_mse)
    temp.append(val_mse)    

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [19]:
all_scores

[13806368768.0,
 18980960256.0,
 13360280576.0,
 15514058752.0,
 12047620096.0,
 12184570880.0,
 16293819392.0,
 16030162944.0,
 16110180352.0,
 17616619520.0]

In [20]:
np.mean(all_scores)

15194464153.6

In [21]:
def build_model(): 
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))
    model.add(keras.layers.Dense(64, activation='selu'))
    model.add(keras.layers.Dense(64, activation='selu'))
    model.add(keras.layers.Dense(64, activation='selu'))
    model.add(keras.layers.Dense(64, activation='selu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
    
    return model

In [22]:
k = 10
num_val = len(X_train) // k
num_epochs = 300
all_scores = []
temp = []

for i in range(k):
    print('processing fold #', i)
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)
    
    model = build_model()
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, 
              verbose=0)

    val_mse, val_mae = model.evaluate(X_val, y_val, verbose=0)
    all_scores.append(val_mse)
    temp.append(val_mse)    

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [23]:
all_scores

[12640817152.0,
 15923390464.0,
 13757606912.0,
 17048558592.0,
 13980404736.0,
 12524831744.0,
 13530193920.0,
 12817385472.0,
 15166085120.0,
 14198850560.0]

In [24]:
np.mean(all_scores)

14158812467.2

In [25]:
def build_model(): 
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))
    model.add(keras.layers.Dense(300, activation='selu'))
    model.add(keras.layers.Dense(200, activation='selu'))
    model.add(keras.layers.Dense(100, activation='selu'))
    model.add(keras.layers.Dense(64, activation='selu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    
    return model

In [26]:
k = 10
num_val = len(X_train) // k
num_epochs = 300
all_scores = []
temp = []

for i in range(k):
    print('processing fold #', i)
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)
    
    model = build_model()
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, 
              verbose=0)

    val_mse, val_mae = model.evaluate(X_val, y_val, verbose=0)
    all_scores.append(val_mse)
    temp.append(val_mse)    

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [27]:
all_scores

[11844591616.0,
 14450809856.0,
 13823792128.0,
 25407162368.0,
 11603164160.0,
 14355822592.0,
 16811030528.0,
 70122725376.0,
 15873592320.0,
 16535623680.0]

In [28]:
np.mean(all_scores)

21082831462.4